# Initialisation

Load, clean, and edit data as required

In [1]:
# Import required libraries
import pandas as pd
from pathlib import Path

In [2]:
# Import required trading data
trading_df = pd.read_csv(
    Path("Data/googl_data.csv"), 
    index_col="Date", 
    infer_datetime_format=True, 
    parse_dates=True
)

# Display sample data
trading_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-04-07,845.000000,845.880005,837.299988,842.099976,842.099976,1111600
2017-04-10,841.539978,846.739990,840.789978,841.700012,841.700012,1046200
2017-04-11,841.700012,844.630005,834.599976,839.880005,839.880005,974300
2017-04-12,838.460022,843.719971,837.590027,841.460022,841.460022,1135800
2017-04-13,841.039978,843.729980,837.849976,840.179993,840.179993,1073700


In [3]:
# Check for nulls

trading_df.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [4]:
# Calculate the daily returns using the closing prices and the pct_change function
trading_df["actual_returns"] = trading_df["Close"].pct_change()

# Drop all NaN values from the DataFrame
trading_df = trading_df.dropna()

# Review the DataFrame
display(trading_df.head())
display(trading_df.tail())

,Open,High,Low,Close,Adj Close,Volume,actual_returns
Date,,,,,,,
2017-04-10,841.539978,846.739990,840.789978,841.700012,841.700012,1046200,-0.000475
2017-04-11,841.700012,844.630005,834.599976,839.880005,839.880005,974300,-0.002162
2017-04-12,838.460022,843.719971,837.590027,841.460022,841.460022,1135800,0.001881
2017-04-13,841.039978,843.729980,837.849976,840.179993,840.179993,1073700,-0.001521
2017-04-17,841.380005,855.640015,841.030029,855.130005,855.130005,1049100,0.017794


,Open,High,Low,Close,Adj Close,Volume,actual_returns
Date,,,,,,,
2022-03-31,2841.060059,2842.159912,2780.489990,2781.350098,2781.350098,1876200,-0.020227
2022-04-01,2790.000000,2809.419922,2766.149902,2803.010010,2803.010010,1296700,0.007788
2022-04-04,2807.169922,2874.239990,2806.209961,2859.429932,2859.429932,1297100,0.020128
2022-04-05,2857.379883,2859.810059,2807.649902,2811.820068,2811.820068,1070900,-0.016650
2022-04-06,2775.000000,2787.209961,2710.340088,2730.959961,2730.959961,1621000,-0.028757


# Define X values (features)

For this example we will use short and long SMA windows of the close price in our data

In [5]:
# Define a window size of 4
short_window = 4

# Create an SMA that uses short_window, and assign it to a new column named “sma_fast”
trading_df["sma_fast"] = trading_df["Close"].rolling(window=short_window).mean()

In [6]:
# Define a window size of 100
long_window = 100

# Create an SMA that uses long_window, and assign it to a new columns named “sma_slow”
trading_df["sma_slow"] = trading_df["Close"].rolling(window=long_window).mean()

In [7]:
# Drop the NaNs using dropna()
trading_df = trading_df.dropna()

In [8]:
# Assign a copy of the `sma_fast` and `sma_slow` columns to a new DataFrame called `X`
# Shift X values 1 day forward to predict next day prices
X = trading_df[["sma_fast", "sma_slow"]].shift().dropna().copy()

# Display sample data
display(X.head())
display(X.tail())

,sma_fast,sma_slow
Date,,
2017-08-31,934.502502,945.779905
2017-09-01,940.687500,946.915305
2017-09-05,946.652496,948.036404
2017-09-06,948.084991,949.036604
2017-09-07,947.682495,950.055004


,sma_fast,sma_slow
Date,,
2022-03-31,2837.862549,2794.726108
2022-04-01,2824.835083,2792.769209
2022-04-04,2818.310059,2790.993108
2022-04-05,2820.640015,2789.803706
2022-04-06,2813.902527,2788.743206


# Define Y (Target)

In this example we use a trading signal that throws a long when actual returns are positive (when the price is going up) and a short when actual returns are negative

Here we're using 1 and 0 so we can later diff the file to create actual trade signals. As is these signals just indicate whether the price is going up or down

In [9]:
# Create a new column in the `trading_df` called "signal" setting its value to zero.
trading_df["signal"] = 0.0

In [10]:
# Create the signal to buy
trading_df.loc[(trading_df["actual_returns"] >= 0), "signal"] = 1

In [11]:
# Create the signal to sell
trading_df.loc[(trading_df["actual_returns"] < 0), "signal"] = 0

In [12]:
# Copy the new "signal" column to a new Series called `y`.
y = trading_df["signal"].copy()

# Split training and testing data

We need to do this manually since our data is sequential and we can't have splitting functions randomise data

The offset will determine how big the training/testing datasets will be. We will use 12 months for now

In [13]:
# Import required libraries
from pandas.tseries.offsets import DateOffset

In [14]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2017-08-31 00:00:00


In [15]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=12)

# Display the training end date
print(training_end)

2018-08-31 00:00:00


In [16]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [17]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Scale Data

In [18]:
# Import required libraries
from sklearn.preprocessing import StandardScaler

In [19]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create the model

Here we are using an svc model which is a classifier from sklearn

In [20]:
# Import the SVM model
from sklearn import svm
from sklearn.metrics import classification_report

In [21]:
# Create the classifier model
svm_model = svm.SVC()
 
# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# Evaluate and backtest the model

We evaluate the model's training data to see how well its training went, then predict using the model and testing data, and run some metrics

In [22]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display the report
print(training_report)

              precision    recall  f1-score   support

         0.0       0.56      0.39      0.46       117
         1.0       0.58      0.74      0.65       136

    accuracy                           0.58       253
   macro avg       0.57      0.56      0.56       253
weighted avg       0.57      0.58      0.56       253



In [23]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [24]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

              precision    recall  f1-score   support

         0.0       0.48      0.34      0.40       413
         1.0       0.56      0.70      0.62       493

    accuracy                           0.53       906
   macro avg       0.52      0.52      0.51       906
weighted avg       0.52      0.53      0.52       906



In [25]:
# This function creates backtesting data using a df of signals indicating price increase or decrease, and an initial capital and trade_size
# We will produce proper trading signals out of them to calculate portfolio change over time, total portfolio value etc.
# Output will be a copy of the df with position changes, holding value, cash on hand, total portfolio value, and daily and cumulative returns
# We will assume input df is a two column df with signal data, close data, and a datetime index
def create_backtesting_data(signal_df, capital, trade_size):
    
    # Grab column name just in case the name is weird
    signal_column = signal_df.columns[0]
    
    # Force the initial position to be 0 otherwise a list of 1s will not be recognised when we run the diff
    signal_df.iloc[0,0] = 0

    # Diff the signal data to get proper trade signals
    signal_df[signal_column] = signal_df[signal_column].diff().dropna()

    # Get position changes by combining trade_size and trade signal
    signal_df["position"] = signal_df[signal_column] * trade_size

    # Get holdings by combining cumulative position and close
    signal_df["holdings"] = signal_df["position"].cumsum() * signal_df["close"]
    
    # Get cumualtive cash on hand by removing cumulative share value from capital
    signal_df["portfolio cash"] = capital - (signal_df["close"] * signal_df["position"]).cumsum()
    
    # Get total portfolio size by adding holdings and liquid
    signal_df["portfolio total"] = (signal_df["portfolio cash"] + signal_df["holdings"])
    
    # Calculate daily returns
    signal_df["portfolio daily returns"] = signal_df["portfolio total"].pct_change()
    
    # Calculate cumulative returns
    signal_df["portfolio cumulative returns"] = (1 + signal_df["portfolio daily returns"]).cumprod() - 1
    
    return signal_df

In [ ]:
# This function is designed to work with the data generated from the create_backtesting_data() function
# We produce the following backtesting metrics: annualized return, cumulative returns, annualised volatility, sharpe ratio, sortino ratio
def create_backtesting_metrics(backtesting_df):
    # Create the list of the metric names
    metrics = ["Annualized Return",
               "Cumulative Returns",
               "Annual Volatility",
               "Sharpe Ratio",
               "Sortino Ratio"]

    # Create a list that holds the column name
    columns = ["Backtest"]

    # Initialize the DataFrame with index set to evaluation metrics and columns
    metrics_df = pd.DataFrame(index=metrics, columns=columns)
    
    # Calculate the Annualized return metric
    metrics_df.loc['Annualized Return'] = backtesting_df["portfolio daily returns"].mean() * 252

    # Calculate the Cumulative returns metric
    metrics_df.loc['Cumulative Returns'] = backtesting_df["portfolio cumulative returns"][-1]

    # Calculate the Annual volatility metric
    metrics_df.loc['Annual Volatility'] = backtesting_df["portfolio daily returns"].std() * np.sqrt(252)
    
    # Calculate the Sharpe ratio
    metrics_df.loc['Sharpe Ratio'] = metrics_df.loc['Annualized Return']/metrics_df.loc['Annual Volatility']

    # Calculate the Sortino ratio
    # Start by calculating the downside return values

    # Create a DataFrame that contains the Portfolio Daily Returns column
    sortino_ratio_df = pd.DataFrame(backtesting_df["portfolio daily returns"])

    # Create a column to hold downside return values
    sortino_ratio_df.loc[:,'Downside Returns'] = 0

    # Find Portfolio Daily Returns values less than 0, 
    # square those values, and add them to the Downside Returns column
    sortino_ratio_df.loc[sortino_ratio_df["portfolio daily returns"] > 0, "Downside Returns"] = sortino_ratio_df["portfolio daily returns"]**2

    # Calculate the annualized return value
    annualized_return = sortino_ratio_df["portfolio daily returns"].mean() * 252

    # Calculate the annualized downside standard deviation value
    downside_standard_deviation = np.sqrt(sortino_ratio_df["Downside Returns"].mean()) * np.sqrt(252)

    # Divide the annualized return value by the downside standard deviation value
    sortino_ratio = annualized_return/downside_standard_deviation

    # Add the Sortino ratio to the evaluation DataFrame
    metrics_df.loc['Sortino Ratio'] = sortino_ratio
    
    return metrics_df


In [34]:
# Initialise capital as 10000
capital = 50000

# Trade shares in 500 share batches
trade_size = 500


In [35]:
# Create df with signal data of training period
# To make the above function work the df will need to be in the following format:
training_evaluation_df = pd.DataFrame({"training signals": training_signal_predictions,}, index = y_train.index)

# Get timeframe for training data
start_date = training_evaluation_df.index[0]
end_date = training_evaluation_df.index[-1]

training_evaluation_df["close"] = trading_df.loc[start_date:end_date, "Close"]

training_evaluation_df.head()

In [36]:
# Run backtesting function
training_evaluation_df = create_backtesting_data(training_evaluation_df, capital, trade_size)

training_evaluation_df.tail()

,training signals,close,position,holdings,portfolio cash,portfolio total,portfolio daily returns,portfolio cumulative returns
Date,,,,,,,,
2017-08-31,NaN,955.239990,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-01,1.0,951.989990,500.0,475994.995117,-425994.995117,50000.000000,NaN,NaN
2017-09-05,0.0,941.479980,0.0,470739.990234,-425994.995117,44744.995117,-0.105100,-0.1051
2017-09-06,0.0,942.020020,0.0,471010.009766,-425994.995117,45015.014648,0.006035,-0.0997
2017-09-07,0.0,949.890015,0.0,474945.007324,-425994.995117,48950.012207,0.087415,-0.0210


In [ ]:
metrics_df = create_backtesting_metrics(training_evaluation_df)

metrics_df.tail()